In [1]:
import base64
import datetime
import hmac
import time
import urllib.request
import requests
from dotenv import load_dotenv
import os
import json
from hashlib import sha1 as sha

In [2]:
load_dotenv("local.env")

True

In [3]:
host = "http://api.velocityweather.com/v1"
access_key = os.getenv("BARON_KEY")
if access_key is None:
    raise ValueError("Missing BARON_KEY in local.env file")
access_key_secret = os.getenv("BARON_SECRET")
if access_key_secret is None:
    raise ValueError("Missing BARON_SECRET in local.env file")

In [4]:
def sign(string_to_sign, secret):
    hmac_sha1 = hmac.new(
        # Convert secret to bytes using UTF-8
        secret.encode('utf-8'),
        # Convert input string to bytes using UTF-8
        string_to_sign.encode('utf-8'),
        sha                        # Use SHA1 hash algorithm
    )

    # Get the binary digest
    hmac_digest = hmac_sha1.digest()

    # Encode to base64
    base64_encoded = base64.b64encode(hmac_digest).decode('utf-8')

    # Replace characters to make URL safe
    signature = base64_encoded.replace('/', '_').replace('+', '-')

    return signature


def sign_request(url, key, secret):
    """ Returns signed url
    """

    ts = str(int(time.time()))
    sig = sign(key + ":" + ts, secret)
    q = '?' if url.find("?") == -1 else '&'
    url += "%ssig=%s&ts=%s" % (q, sig, ts)
    return url


In [5]:
def request_wms(product, product_config, image_size_in_pixels, image_bounds):
    """
    Requests a WMS image and saves it to disk in the current directory.
    @param product: The product code, such as 'C39-0x0302-0'
    @param product_config: The product configuration, such as 'Standard-Mercator' or 'Standard-Geodetic'.
    @param image_size_in_pixels: The image width and height in pixels, such as [1024, 1024].
    @param image_bounds: The bounds of the image. This value has several caveats, depending
        on the projection being requested.
        A. If requesting a Mercator (EPSG:3857) image:
            1. The coordinates must be in meters.
            2. The WMS 1.3.0 spec requires the coordinates be in this order [xmin, ymin, xmax, ymax]
            3. As an example, to request the whole world, you would use [-20037508.342789244, -20037508.342789244, 20037508.342789244, 20037508.342789244].
               Because this projection stretches to infinity as you approach the poles, the ymin and ymax values
               are clipped to the equivalent of -85.05112877980659 and 85.05112877980659 latitude, not -90 and 90 latitude,
               resulting in a perfect square of projected meters.
        B. If requesting a Geodetic (EPSG:4326) image:
            1. The coordinates must be in decimal degrees.
            2. The WMS 1.3.0 spec requires the coordinates be in this order [lat_min, lon_min, lat_max, lon_max].
            3. As an example, to request the whole world, you would use [-90, -180, 90, 180].

    Theoretically it is possible to request any arbitrary combination of image_size_in_pixels and image_bounds,
    but this is not advisable and is actually discouraged. It is expected that the proportion you use for
    image_width_in_pixels/image_height_in_pixels is equal to image_width_bounds/image_height_bounds. If this is
    not the case, you have most likely done some incorrect calculations. It will result in a distorted (stretched
    or squished) image that is incorrect for the requested projection. One fairly obvious sign that your
    proportions don't match up correctly is that the image you receive from your WMS request will have no
    smoothing (interpolation), resulting in jaggy or pixelated data.
    """

    # We're using the TMS-style product instances API here for simplicity. If you
    # are using a standards-compliant WMS client, do note that we also provide a
    # WMS-style API to retrieve product instances which may be more suitable to your
    # needs. See our documentation for details.

    # For this example, we use the optional parameter "page_size" to limit the
    # list of product instances to the most recent instance.
    meta_url = '{}/{}/meta/tiles/product-instances/{}/{}?page_size=1'.format(
        host, access_key, product, product_config)
    meta_url = sign_request(meta_url, access_key, access_key_secret)

    # request = urllib.Request(meta_url)
    try:
        response = urllib.request.urlopen(meta_url)
    except urllib.error.HTTPError as e:
        print('HTTP status code:', e.code)
        print('content:')
        print(e.read())
        return
    assert response.code == 200

    # Decode the product instance response and get the most recent product instance time,
    # to be used in the WMS image request.
    content = json.loads(response.read())
    product_instance_time = content[0]['time']

    # WMS uses EPSG codes, while our product configuration code uses 'Geodetic' or
    # 'Mercator'. We map between the two here to prepare for the WMS CRS query parameter.
    epsg_code = 'EPSG:4326' if product_config.endswith(
        '-Geodetic') else 'EPSG:3857'

    # Convert the image bounds to a comma-separated string.
    image_bounds = ','.join(str(x) for x in image_bounds)

    wms_url = '{}/{}/wms/{}/{}?VERSION=1.3.0&SERVICE=WMS&REQUEST=GetMap&CRS={}&LAYERS={}&BBOX={}&WIDTH={}&HEIGHT={}'.format(
        host,
        access_key,
        product,
        product_config,
        epsg_code,
        product_instance_time,
        image_bounds,
        image_size_in_pixels[0],
        image_size_in_pixels[1]
    )
    wms_url = sign_request(wms_url, access_key, access_key_secret)
    print(wms_url)

    return(wms_url)

def request_wms_image(product, product_config, image_size_in_pixels, image_bounds):
    """ 
    Requests a WMS image and saves it to disk in the current directory.
    """
    wms_url = request_wms(product, product_config, image_size_in_pixels, image_bounds)
    if wms_url is None:
        print('Failed to get WMS URL')
        return
    try:
        response = urllib.request.urlopen(wms_url)
    except urllib.error.HTTPError as e:
        print('HTTP status code:', e.code)
        print('content:')
        print(e.read())
        return
    assert response.code == 200

    content = response.read()
    filename = './wms_img_{}_{}.png'.format(product, product_config)
    print('Read {} bytes, saving as {}'.format(len(content), filename))
    with open(filename, 'wb') as f:
        f.write(content)


In [6]:
def request_wms_capabilities(product, product_config):
    capabilities_url = f"{host}/{access_key}/wms/{product}/{product_config}?SERVICE=WMS&REQUEST=GetCapabilities"
    print(f"capabilities_url={capabilities_url}")
    capabilities_url = sign_request(capabilities_url, access_key, access_key_secret)
    print(f"capabilities_url={capabilities_url}")
    try:
        response = urllib.request.urlopen(capabilities_url)
    except urllib.error.HTTPError as e:
        print('HTTP status code:', e.code)
        print('content:')
        print(e.read())
        return
    assert response.code == 200
    content = response.read()
    filename = './wms_capabilities_{}_{}.xml'.format(product, product_config)
    print('Read {} bytes, saving as {}'.format(len(content), filename))
    with open(filename, 'wb') as f:
        f.write(content)
    print(f"Saved capabilities to {filename}")
    

In [7]:
texas_bound_box = [-106.645646, 36.500704, -93.508292, 25.837164]
arkansas_bound_box = [-94.724121, 32.512896, -89.428711, 36.576877]
usa_bound_box = [-125.859375, 25.618963, -63.193359, 49.378416]
temp_bound_box = [-106.645646, 36.500704, -105.808292, 35.837164]
whole_world_bound_box = [-90, -180, 90, 180]
tx_panhandle_bound_box = [-103.05, 34.65, -99.99, 36.53]

In [8]:
import leafmap

In [9]:
hc_lat = 29.85722627925764
hc_long = -95.39202050686889
m = leafmap.Map(center=[hc_lat, hc_long], zoom=7,
                epsg="4326")

In [10]:
# product = 'fire-tracker-us'
product = 'C39-0x0302-0' # 'north-american-radar' # 'fspc-day2-outlook' #'C39-0x0355-0' # 
product_config = 'Standard-Geodetic' # 'Standard-Mercator' # 
image_size_in_pixels = [2048, 2048]
image_bounds = usa_bound_box # texas_bound_box

request_wms_image(product, product_config, image_size_in_pixels, usa_bound_box)
request_wms_capabilities(product, product_config)


http://api.velocityweather.com/v1/XMFbRooKrTYP/wms/C39-0x0302-0/Standard-Geodetic?VERSION=1.3.0&SERVICE=WMS&REQUEST=GetMap&CRS=EPSG:4326&LAYERS=2025-05-15T21:20:30Z&BBOX=-125.859375,25.618963,-63.193359,49.378416&WIDTH=2048&HEIGHT=2048&sig=tBm3GV5HiZxCDsJdNIxSZXcpXaA=&ts=1747344046
Read 5193 bytes, saving as ./wms_img_C39-0x0302-0_Standard-Geodetic.png
capabilities_url=http://api.velocityweather.com/v1/XMFbRooKrTYP/wms/C39-0x0302-0/Standard-Geodetic?SERVICE=WMS&REQUEST=GetCapabilities
capabilities_url=http://api.velocityweather.com/v1/XMFbRooKrTYP/wms/C39-0x0302-0/Standard-Geodetic?SERVICE=WMS&REQUEST=GetCapabilities&sig=tBm3GV5HiZxCDsJdNIxSZXcpXaA=&ts=1747344046
Read 5592 bytes, saving as ./wms_capabilities_C39-0x0302-0_Standard-Geodetic.xml
Saved capabilities to ./wms_capabilities_C39-0x0302-0_Standard-Geodetic.xml


In [11]:
# Mercator
image_bounds_mercator = [-20037508.342789244, -20037508.342789244, 20037508.342789244, 20037508.342789244]
# request_wms_image(product, 'Standard-Mercator', image_size_in_pixels, image_bounds_mercator)

In [12]:
wms_url = request_wms(product, product_config, image_size_in_pixels, usa_bound_box)
print(f"wms_url={wms_url}")


http://api.velocityweather.com/v1/XMFbRooKrTYP/wms/C39-0x0302-0/Standard-Geodetic?VERSION=1.3.0&SERVICE=WMS&REQUEST=GetMap&CRS=EPSG:4326&LAYERS=2025-05-15T21:20:30Z&BBOX=-125.859375,25.618963,-63.193359,49.378416&WIDTH=2048&HEIGHT=2048&sig=tBm3GV5HiZxCDsJdNIxSZXcpXaA=&ts=1747344046
wms_url=http://api.velocityweather.com/v1/XMFbRooKrTYP/wms/C39-0x0302-0/Standard-Geodetic?VERSION=1.3.0&SERVICE=WMS&REQUEST=GetMap&CRS=EPSG:4326&LAYERS=2025-05-15T21:20:30Z&BBOX=-125.859375,25.618963,-63.193359,49.378416&WIDTH=2048&HEIGHT=2048&sig=tBm3GV5HiZxCDsJdNIxSZXcpXaA=&ts=1747344046


In [13]:
layer_name = "Baron-NA-RADAR"

m.add_wms_layer(
    url=wms_url,
    layers=layer_name,
    name="Baron Radar",
    format='image/png',
    transparent=True,
    opacity=0.4
)

In [14]:
m

Map(center=[29.85722627925764, -95.39202050686889], controls=(ZoomControl(options=['position', 'zoom_in_text',…